In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from priceanalytics.data import download_df_map, cache_df_map, load_cached_df_map
from priceanalytics.plot import MultiPlot
from priceanalytics import indicators as I
from priceanalytics.backtest import Backtester

from datetime import datetime, date, time

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
tickers = ['AAPL', 'MSFT']
raw_tdfs = download_df_map(tickers, interval=1)
# cache_df_map(raw_tdfs, _dir="cache")


raw AAPL: 26489 datapoints from 2023-09-25 to 2023-11-15
raw MSFT: 22723 datapoints from 2023-09-25 to 2023-11-15


In [4]:
old_tdfs = load_cached_df_map(_dir="cache") # was @ interval = 5
old_dataset = old_tdfs['AAPL'] + old_tdfs['MSFT']

tickers = ['AAPL', 'MSFT', 'AMZN']
new_tdfs = download_df_map(tickers, interval=5)
new_dataset = []
for ticker, dfs in new_tdfs.items():
    new_dataset += dfs
    
dataset = old_dataset + new_dataset

datapts = sum([len(df) for df in dataset])
print("Loaded", len(dataset), "datasets containing", datapts, "datapoints")

raw AAPL: 6633 datapoints from 2023-09-25 to 2023-11-15
raw MSFT: 6069 datapoints from 2023-09-25 to 2023-11-15
raw AMZN: 6589 datapoints from 2023-09-25 to 2023-11-15
Loaded 129 datasets containing 23439 datapoints


In [47]:
import priceanalytics.backtest as bt
 
def test_and_strat(total, _len, sig_len, stop_loss=-10): #-1000% will never happen (hopefully)
    tpr = 1
    incrs = []
    
    buys, sells = [], []
    for n, df in enumerate(total):

        df = I.add_square_bounds_osc(df, _len=_len, sig_len=sig_len)
        df = df.iloc[_len:]

        with bt.Backtester(df, verbose=False) as b:
            last_buy = 0
            for i, fr in b:
                
                if b.crossover("sbosc_bull", "sbosc_bear"):
                    if not b.in_position: #should probably do this check automatically
                        b.buy()
                        last_buy = b.price()
                
                elif b.crossover("sbosc_signal", "sbosc_bull"):
                    if b.in_position: #should also probably do this check automatically
                        b.sell()
                    
                if b.in_position: # stop loss calculation
                    fallen = (b.price() - last_buy) / last_buy
                    if fallen < stop_loss:
                        b.sell()
                        inpos=False
            
            buys.append(b.buys)
            sells.append(b.sells)
            
            incr = b.results(1)
            incrs.append(incr)
            tpr *= (1 + incr)
    return tpr - 1, incrs, (buys, sells)


In [66]:
from priceanalytics.data import combine_dataset, calc_returns_homogenous

validation_dataset = combine_dataset(download_df_map(['AMZN'], interval=1))
    
wds_ret = calc_returns_homogenous(validation_dataset)

#tpr, incrs = test_and_strat(dataset, 14, 21)    
tpr, incrs, trades = test_and_strat(validation_dataset, 14, 21, stop_loss=-.03)    

# for i, df in enumerate(validation_dataset):
#     df = df.iloc[14:] # [_len:]
#     buys, sells = trades[0][i], trades[1][i]
#     with MultiPlot(1, size=(6, 3)) as m:
#         print(f'returns {bt.perc_ret(incrs[i])}%')
#         m.ohlc(df)
#         #plt.scatter(buys.keys(), buys.values(), color="#00FF00", marker="^")
#         for b in buys.keys():  
#             plt.axvline(x=b, color="#00FF00")
#         #plt.scatter(sells.keys(), sells.values(), color="#FF0000", marker="v")
#         for s in sells.keys():
#             plt.axvline(x=s, color="#FF0000")

print('')
bt.print_test_results(incrs)

KeyboardInterrupt: 

## notes

After a short time testing this out I noticed three things.
1. If neither line is above the signal line then it could be a choppy market.
2. If one line is almost on top of the signal line or barely above it then it could also be chop and sometimes it lasts a little while. Waiting until it is clearly above the signal line might work better.
3. If a position is taken and the bull/bear line goes below the signal line then it could be used as an exit. However it could also mean the market is going to have a bit of chop or slight retracement before continuing if the opposite line doesn't cross above the signal line.

So far it looks phenomenal

Oh I forgot to add, if in a position and the line stacks on top of the signal line then it can be an exit or everything else I listed in point 3 of my previous comment.